In [1]:
from __future__ import print_function

import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import BatchNormalization as BN
from keras.layers import GaussianNoise as GN
from tensorflow.keras.optimizers import SGD
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler as LRS
from keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam, Adagrad

ImportError: cannot import name 'RMSprop' from 'keras.optimizers' (C:\Users\Usuario\anaconda3\envs\rasa\lib\site-packages\keras\optimizers.py)

In [ ]:
batch_size = 128
num_classes = 10
epochs = 250

In [ ]:
#### LOAD AND TRANSFORM
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_test /= 255

print(x_train.shape)
print(x_test.shape)


170498071/170498071 [==============================] - 13s 0us/step
(50000, 32, 32, 3)
(10000, 32, 32, 3)


In [ ]:
y_train = keras.utils.np_utils.to_categorical(y_train, num_classes)
y_test = keras.utils.np_utils.to_categorical(y_test, num_classes)

In [ ]:
datagen = ImageDataGenerator(
  featurewise_center=True,
  featurewise_std_normalization=True,
  width_shift_range=0.2,
  height_shift_range=0.2,
  rotation_range=20,
  zoom_range=[1.0,1.2],
  horizontal_flip=True)

In [ ]:
datagen.fit(x_train)

testdatagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,)

testdatagen.fit(x_train)

In [ ]:
## DEF A BLOCK CONV + BN + GN + MAXPOOL
def CBGN(model,filters,ishape=0):
  if (ishape!=0):
    model.add(Conv2D(filters, (3, 3), padding='same',
                 input_shape=ishape))
  else:
    model.add(Conv2D(filters, (3, 3), padding='same'))
  
  #model.add(BN())
  #model.add(GN(0.3))
  #model.add(Activation('relu'))
  #model.add(MaxPooling2D(pool_size=(2, 2)))
  
  model.add(BN())
  model.add(GN(0.1))
  model.add(Activation('relu'))

  model.add(Conv2D(filters, (3, 3), padding='same'))
  model.add(BN())
  model.add(GN(0.1))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  
  return model


In [ ]:
## DEF NN TOPOLOGY  
model = Sequential()

model=CBGN(model,32,x_train.shape[1:])
model=CBGN(model,64)
model=CBGN(model,128)
model=CBGN(model,256)
model=CBGN(model,512)

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))

model.add(Dense(num_classes))
model.add(Activation('softmax'))


model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 32)       128       
 ormalization)                                                   
                                                                 
 gaussian_noise (GaussianNoi  (None, 32, 32, 32)       0         
 se)                                                             
                                                                 
 activation (Activation)     (None, 32, 32, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 batch_normalization_1 (Batc  (None, 32, 32, 32)       1

In [ ]:
## OPTIM AND COMPILE
opt = Adam(learning_rate=1e-3)
#opt = Adagrad
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, cooldown=1,
                              patience=10, min_lr=0.00005)

In [ ]:
## TRAINING with DA and LRA
history=model.fit(datagen.flow(x_train, y_train,batch_size=batch_size),
                            steps_per_epoch=len(x_train) / batch_size, 
                            epochs=epochs,
                            #validation_data=(x_test, y_test),
                            validation_data=testdatagen.flow(x_test, y_test),
                            callbacks=[reduce_lr],
                            verbose=1)

Epoch 1/250
390/390 [==============================] - 52s 107ms/step - loss: 1.5774 - accuracy: 0.4214 - val_loss: 1.8186 - val_accuracy: 0.4363 - lr: 0.0010
Epoch 2/250
390/390 [==============================] - 41s 104ms/step - loss: 1.1343 - accuracy: 0.5940 - val_loss: 1.1251 - val_accuracy: 0.6138 - lr: 0.0010
Epoch 3/250
390/390 [==============================] - 41s 105ms/step - loss: 0.9459 - accuracy: 0.6669 - val_loss: 1.1457 - val_accuracy: 0.6049 - lr: 0.0010
Epoch 4/250
390/390 [==============================] - 39s 101ms/step - loss: 0.8320 - accuracy: 0.7087 - val_loss: 0.7985 - val_accuracy: 0.7229 - lr: 0.0010
Epoch 5/250
390/390 [==============================] - 41s 105ms/step - loss: 0.7536 - accuracy: 0.7391 - val_loss: 0.9503 - val_accuracy: 0.7004 - lr: 0.0010
Epoch 6/250
390/390 [==============================] - 40s 103ms/step - loss: 0.6951 - accuracy: 0.7604 - val_loss: 0.8144 - val_accuracy: 0.7339 - lr: 0.0010
Epoch 7/250
390/390 [=========================